In [1]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics, model_selection
import keras_tuner

2025-03-22 12:09:31.067766: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-03-22 12:09:39.183854: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2025-03-22 12:09:39.183910: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2025-03-22 12:09:52.710492: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2025-

### Load the data

In [2]:
# remove val data as stratified kfold is performed automatically
X_train = np.load("data/X_train.npy")
y_train = np.load("data/y_train.npy")
X_test = np.load("data/X_test.npy")
y_test = np.load("data/y_test.npy")

In [3]:
X_train_SMOTE = np.load("data/X_train_SMOTE.npy")
y_train_SMOTE = np.load("data/y_train_SMOTE.npy")
X_test_SMOTE = np.load("data/X_test_SMOTE.npy")
y_test_SMOTE = np.load("data/y_test_SMOTE.npy")

### Model training

In [5]:
def build_model(hp):
    model = RandomForestClassifier(
        n_estimators=hp.Choice("n_estimators", [50, 100, 150]),
        random_state=42,
        min_samples_split=hp.Choice("min_samples_split", [2, 5]),
        min_samples_leaf=hp.Choice("min_samples_leaf", [2, 5])
        )
    return model

tuner = keras_tuner.tuners.SklearnTuner(
    oracle=keras_tuner.oracles.BayesianOptimizationOracle(
         objective=keras_tuner.Objective('score', 'max'),
         max_trials=10),
    hypermodel=build_model,
    scoring=metrics.make_scorer(metrics.accuracy_score),
    #cv=model_selection.StratifiedKFold(5),
    directory='.',
    project_name='random_forest_search'
)
    

In [6]:
tuner.search(X_train, y_train)

TypeError: SklearnTuner.search() got an unexpected keyword argument 'validation_split'